## Accessing DM-EFD data


In this notebook, we demonstrate simple usage of the LSST EFD python client.  The client is pip installable via, e.g., `pip install lsst-efd-client`.

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

You will need `astropy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

Credentials are taken from a special file on disk.  See the readme [here](https://pypi.org/project/lsst-efd-client/) or ping `@ksk` on slack.

### Import modules and construct the client

In [ ]:
from lsst_efd_client import EfdClient, resample

In [ ]:
efd_client = EfdClient('int_efd')

It's easy to query for topics or fields in a specific topic.

In [ ]:
topics = await efd_client.get_topics()
topics[:15]

In [ ]:
fields = await efd_client.get_fields(topics[0])
fields

### Query for data

Times need to be specified in TAI and with `astropy` time-like objects.

In [ ]:
start = Time('2019-12-11T05:20:00', scale='tai')
end = TimeDelta(60, format='sec', scale='tai')

Query for some high cadence data.  This assumes 100 samples per message.

In [ ]:
measured_azel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.mount_AzEl_Encoders', ['azimuthCalculatedAngle', 'elevationCalculatedAngle'], start, end)
measured_azel_vel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.measuredMotorVelocity', ['azimuthMotor1Velocity', 'azimuthMotor2Velocity', 'elevationMotorVelocity'], start, end)
commanded_azel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.trajectory', ['azimuth', 'azimuthVelocity', 'elevation', 'elevationVelocity'], start, end)

Plot the data.

In [ ]:
p = figure(x_axis_type='datetime', plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_azel.index.values, y=commanded_azel['azimuth'], color='black', line_width=2, legend_label='Commanded Az')
p.line(x=measured_azel.index.values, y=measured_azel['azimuthCalculatedAngle'], color='lightblue', line_width=2, legend_label='Computed Az')
p.extra_y_ranges = {'Velocity': Range1d(start=-2.5, end=2.5)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='Measured Az Velocity Motor 1')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='Measured Az Velocity Motor 2')
p.line(x=commanded_azel.index.values, y=commanded_azel['azimuthVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='Commanded Az Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

There is a convenience function to resample two time series onto each other.  This allows plotting/analysis of time series that are not sampled on the same times.

In [ ]:
interp = resample(measured_azel, commanded_azel)
ratio = interp['elevationCalculatedAngle']/interp['elevation']

In [ ]:
p = figure(x_axis_type='datetime', y_range=(0.99999, 1.00001), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Ratio of measured to commanded elevation"
p.xaxis.axis_label = "Time"
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='Measured El Velocity Motor')
p.line(x=commanded_azel.index.values, y=commanded_azel['elevationVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='Commanded El Velocity')
p.line(x=ratio.index.values, y=ratio.values, color='black', line_width=2, legend_label='ratio', alpha=0.3)
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)